- https://google.github.io/adk-docs/deploy/agent-engine/
- https://cloud.google.com/python/docs/reference/vertexai/latest/vertexai.preview.reasoning_engines.ReasoningEngine#vertexai_preview_reasoning_engines_ReasoningEngine_list

# install packages

In [2]:
!pip install -U google-genai google-adk google-cloud-aiplatform

  Using cached google_adk-1.1.1-py3-none-any.whl.metadata (9.5 kB)
Using cached google_adk-1.1.1-py3-none-any.whl (1.2 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 96.0 MB/s eta 0:00:00
  Attempting uninstall: google-genai
    Found existing installation: google-genai 1.16.1
    Uninstalling google-genai-1.16.1:
      Successfully uninstalled google-genai-1.16.1
  Attempting uninstall: google-cloud-aiplatform
    Found existing installation: google-cloud-aiplatform 1.95.0
    Uninstalling google-cloud-aiplatform-1.95.0:
      Successfully uninstalled google-cloud-aiplatform-1.95.0
  Attempting uninstall: google-adk
    Found existing installation: google-adk 0.5.0
    Uninstalling google-adk-0.5.0:
      Successfully uninstalled google-adk-0.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
vertexai 1.71.1 requires google-cloud-aiplatform[all]==

# develop agent (Simple)

In [3]:
! pip list | grep -e adk -e aiplatform

google-adk                               1.1.1
google-cloud-aiplatform                  1.71.1


In [4]:
!gsutil ls

gs://255766800726_us_central1_import_content/
gs://255766800726_us_central1_import_content_with_faq_csv/
gs://255766800726_us_import_content_with_faq_csv/
gs://255766800726_us_import_product/
gs://255766800726_us_import_user_event/
gs://avatar-bucket-notebook/
gs://cloud-ai-platform-67039ab0-1236-4b5e-8e7c-9bb8efa07266/
gs://dataproc-staging-us-central1-255766800726-grrfr5mh/
gs://genai-experiments-sm/
gs://hp-books-my-project-0004-346516-pytor-asia-southeast1/
gs://imagen-style-training/
gs://masked-audio-hk2/
gs://my-bk-my-project-0004--pytorch112kag-us-central1/
gs://my-my-project-0004--pytorch112kag-us-central1/
gs://my-project-0004-346516/
gs://my-project-0004-346516-asia-notebooks/
gs://my-project-0004-346516-darrylvideoragdemo-us-central1/
gs://my-project-0004-346516-jingletest1/
gs://my-project-0004-346516-movielens-data/
gs://my-project-0004-346516-pruclaim/
gs://my-project-0004-346516-sd/
gs://my-project-0004-346516-sd02/
gs://my-project-0004-346516-sd02-us-central1/
gs://my-

In [5]:
import vertexai
from google.adk.agents import Agent

In [6]:
import os

# Cloud project id.
PROJECT_IDS = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_IDS[0]  # @param {type:"string"}

if not PROJECT_ID:
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = "us-central1" # @param {type:"string"}

os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
os.environ["GOOGLE_CLOUD_LOCATION"] = LOCATION
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "TRUE" # Use Vertex AI API

# Print project and location details
print(f"Project ID:", PROJECT_ID)
print(f"Project Region:", LOCATION)


Project ID: my-project-0004-346516
Project Region: us-central1


In [7]:
from google.cloud import storage

client = storage.Client()

GCS_BUCKET_LOCATION = LOCATION
UNIQUE_PREFIX = 'sm' #"<REPLACE_WITH_PREFIX>"

GCS_BUCKET_NAME = f"{PROJECT_ID}-{UNIQUE_PREFIX}"
GCS_BUCKET_URI = f"gs://{GCS_BUCKET_NAME}"

bucket = storage.Bucket(client, GCS_BUCKET_NAME)

if bucket.exists()==False:
    # Create a Cloud Storage Bucket
    !gcloud storage buckets create $GCS_BUCKET_URI --location=$GCS_BUCKET_LOCATION

else:    
    print(f"\n{GCS_BUCKET_NAME} already exists.")
    
def gcs_file(blob_name):
    return bucket.blob(blob_name)



my-project-0004-346516-sm already exists.


In [14]:

STAGING_BUCKET = GCS_BUCKET_URI  #@param{type:"string"}

vertexai.init(
    project=PROJECT_ID,
    location=LOCATION,
    staging_bucket=STAGING_BUCKET,
)

In [9]:
from google.adk.agents import Agent

root_agent = Agent(
    model='gemini-2.0-flash-001',
    name='hotel_agent',
    description='A helpful assistant that answers questions about a specific city focused on Asia',
    instruction='Answer user questions about a specific city related to Asia to the best of your knowledge. Do not answer questions outside of this.',
)

# local testing

In [10]:
from vertexai.preview import reasoning_engines

app = reasoning_engines.AdkApp(
    agent=root_agent,
    enable_tracing=True,
)

In [11]:
session = app.create_session(user_id="u_123")
session

Session(id='383f0f1d-23a1-4ce9-af42-09e6945dd6fb', app_name='default-app-name', user_id='u_123', state={}, events=[], last_update_time=1748791142.9375544)

In [12]:
for event in app.stream_query(
    user_id="u_123",
    session_id=session.id,
    message="whats the best fact about Singapore",
):
  print(event)

{'content': {'parts': [{'text': 'Singapore is one of only three surviving city-states in the world.\n'}], 'role': 'model'}, 'usage_metadata': {'candidates_token_count': 15, 'candidates_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>, 'token_count': 15}], 'prompt_token_count': 61, 'prompt_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>, 'token_count': 61}], 'total_token_count': 76, 'traffic_type': <TrafficType.ON_DEMAND: 'ON_DEMAND'>}, 'invocation_id': 'e-6188a6ed-351b-4a5a-8726-41e1c3749fea', 'author': 'hotel_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'id': 'yjrsNkVb', 'timestamp': 1748791158.3058}


# deploy agent

In [15]:
from vertexai import agent_engines

name = dict(root_agent)['name']

# https://github.com/googleapis/python-aiplatform/blob/main/vertexai/agent_engines/_agent_engines.py
remote_app = agent_engines.create(
    display_name=name,
    agent_engine=root_agent,
    requirements=[
        "google-cloud-aiplatform[adk,agent_engines]",
    ]
)

Deploying google.adk.agents.Agent as an application.
Identified the following requirements: {'pydantic': '2.11.2', 'cloudpickle': '3.0.0', 'google-cloud-aiplatform': '1.83.0'}
The following requirements are missing: {'pydantic', 'cloudpickle'}
The following requirements are appended: {'pydantic==2.11.2', 'cloudpickle==3.0.0'}
The final list of requirements: ['google-cloud-aiplatform[adk,agent_engines]', 'pydantic==2.11.2', 'cloudpickle==3.0.0']
Using bucket my-project-0004-346516-sm
Wrote to gs://my-project-0004-346516-sm/agent_engine/agent_engine.pkl
Writing to gs://my-project-0004-346516-sm/agent_engine/requirements.txt
Creating in-memory tarfile of extra_packages
Writing to gs://my-project-0004-346516-sm/agent_engine/dependencies.tar.gz
Creating AgentEngine
Create AgentEngine backing LRO: projects/255766800726/locations/us-central1/reasoningEngines/1512268292843110400/operations/7124018788806164480
View progress and logs at https://console.cloud.google.com/logs/query?project=my-proj

In [16]:
for agent in agent_engines.list():
  print(agent)

resource name: projects/255766800726/locations/us-central1/reasoningEngines/1512268292843110400
resource name: projects/255766800726/locations/us-central1/reasoningEngines/6201641404842639360
resource name: projects/255766800726/locations/us-central1/reasoningEngines/3168044843343544320
resource name: projects/255766800726/locations/us-central1/reasoningEngines/7354562909449486336
resource name: projects/255766800726/locations/us-central1/reasoningEngines/6651579155114622976
resource name: projects/255766800726/locations/us-central1/reasoningEngines/3636419204590075904
resource name: projects/255766800726/locations/us-central1/reasoningEngines/6382102049286258688
resource name: projects/255766800726/locations/us-central1/reasoningEngines/694055719917322240
resource name: projects/255766800726/locations/us-central1/reasoningEngines/2985543505318772736
Failed to register API methods. Please follow the guide to register the API methods: https://cloud.google.com/vertex-ai/generative-ai/doc

In [17]:
print(remote_app.display_name)

hotel_agent


# remote testing

In [18]:
remote_session = remote_app.create_session(user_id="u_456")
remote_session

{'appName': '1512268292843110400',
 'events': [],
 'state': {},
 'id': '443301647842541568',
 'lastUpdateTime': 1748791363.058443,
 'userId': 'u_456'}

In [19]:
remote_app.list_sessions(user_id="u_456")

{'sessions': [{'appName': '1512268292843110400',
   'events': [],
   'state': {},
   'id': '443301647842541568',
   'lastUpdateTime': 1748791363.058443,
   'userId': 'u_456'}]}

In [20]:
for event in remote_app.stream_query(
    user_id="u_456",
    session_id=remote_session["id"],
    message="tell me a best fact about Singapore airlines and how it won over Quantas",
):
    print(event)

{'content': {'parts': [{'text': 'Singapore Airlines is renowned for its exceptional service and consistently ranks among the world\'s best airlines.\n\nAs for how it "won over" Qantas, that\'s a complex issue. Qantas is a very established airline with a long history and strong brand recognition, particularly in Australia. However, Singapore Airlines has definitely become a major competitor in the international market, known for its superior customer experience and modern fleet.\n'}], 'role': 'model'}, 'usage_metadata': {'candidates_token_count': 82, 'candidates_tokens_details': [{'modality': 'TEXT', 'token_count': 82}], 'prompt_token_count': 70, 'prompt_tokens_details': [{'modality': 'TEXT', 'token_count': 70}], 'total_token_count': 152, 'traffic_type': 'ON_DEMAND'}, 'invocation_id': 'e-183ce025-0632-4c15-bcda-4271b6caa818', 'author': 'hotel_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'id': 'HJNeIXx9', 'timestamp': 1748791496.57071}
